# Training model

In [1]:
## To get some insights
# https://github.com/ceptln/paris-bike-traffic-prediction/tree/main

from pathlib import Path
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from itertools import combinations
import sklearn
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
import lightgbm as lgb
from sklearn.linear_model import Lasso, ElasticNet
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

from submissions.external_data.estimator import _encode_dates, _merge_external_data
import utils.get_data_index_arange as get_data

### Function submission kaggle

In [2]:
def submission_kaggle(model, X_final_test, original_index):
    y_pred = model.predict(X_final_test)
    print(y_pred)
    results = pd.DataFrame(
        dict(
            Id=original_index,
            log_bike_count=y_pred,
        )
    )
    results.to_csv("submission.csv", index=False)

    return results

## Read Data

In [3]:
def add_external_data(X):
    
    X = X.copy()
    
    # merge original data + external data
    merged_X_train_external_DATA = get_data._merge_external_data_weather(X)
    
    # merge original data + external data + holidays
    merged_X_train_external_HOLIDAYS = get_data._merge_holidays_week_end(merged_X_train_external_DATA)
    
    # merge original data + external data + holidays + data COVID
    merged_X_train_external_HOLIDAYS_COVID = get_data._merge_Curfews_lockdowns_COVID(merged_X_train_external_HOLIDAYS)
    merged_X_train_external_HOLIDAYS_COVID = get_data._merge_indicators_COVID(merged_X_train_external_HOLIDAYS_COVID)
    
    # merge original data + external data + holidays + data COVID + data accidents
    merged_X_train_external_HOLIDAYS_COVID_ACCIDENTS = get_data._merge_road_accidents(merged_X_train_external_HOLIDAYS_COVID)
     
    return merged_X_train_external_HOLIDAYS_COVID_ACCIDENTS

In [4]:
# Read data
X_train, y_train = get_data.get_train_data()
X_test, y_test = get_data.get_test_data()
X_final_test = get_data.get_final_test_data()

In [5]:
X_train_plus = add_external_data(X_train)
X_train_plus.head()

,counter_id,counter_name,site_id,site_name,date,counter_installation_date,counter_technical_id,latitude,longitude,ff,...,is_holiday,is_weekend,is_lockdown,is_curfew,hosp,rea,incid_rea,rad,Max_Grav_accidents,Count_accidents
0,100049407-353255860,152 boulevard du Montparnasse E-O,100049407,152 boulevard du Montparnasse,2020-09-01 01:00:00,2018-12-07,Y2H19070373,48.840801,2.333233,1.6,...,False,0,False,False,293,42,3.0,6641,0.0,0.0
30,100049407-353255859,152 boulevard du Montparnasse O-E,100049407,152 boulevard du Montparnasse,2020-09-01 01:00:00,2018-12-07,Y2H19070373,48.840801,2.333233,1.6,...,False,0,False,False,293,42,3.0,6641,0.0,0.0
31,100036719-104036719,18 quai de l'Hôtel de Ville NO-SE,100036719,18 quai de l'Hôtel de Ville,2020-09-01 01:00:00,2017-07-12,Y2H19027732,48.853720,2.357020,1.6,...,False,0,False,False,293,42,3.0,6641,0.0,0.0
32,100036719-103036719,18 quai de l'Hôtel de Ville SE-NO,100036719,18 quai de l'Hôtel de Ville,2020-09-01 01:00:00,2017-07-12,Y2H19027732,48.853720,2.357020,1.6,...,False,0,False,False,293,42,3.0,6641,0.0,0.0
33,100063175-353277233,20 Avenue de Clichy NO-SE,100063175,20 Avenue de Clichy,2020-09-01 01:00:00,2020-07-22,Y2H20073268,48.885290,2.326660,1.6,...,False,0,False,False,293,42,3.0,6641,0.0,0.0


In [6]:
X_test_plus = add_external_data(X_test)
X_test_plus.head()

,counter_id,counter_name,site_id,site_name,date,counter_installation_date,counter_technical_id,latitude,longitude,ff,...,is_holiday,is_weekend,is_lockdown,is_curfew,hosp,rea,incid_rea,rad,Max_Grav_accidents,Count_accidents
0,100049407-353255860,152 boulevard du Montparnasse E-O,100049407,152 boulevard du Montparnasse,2021-08-10 01:00:00,2018-12-07,Y2H19070373,48.840801,2.333233,1.9,...,False,0,False,False,284,84,6.0,21167,0.0,0.0
31,100049407-353255859,152 boulevard du Montparnasse O-E,100049407,152 boulevard du Montparnasse,2021-08-10 01:00:00,2018-12-07,Y2H19070373,48.840801,2.333233,1.9,...,False,0,False,False,284,84,6.0,21167,0.0,0.0
32,100036719-104036719,18 quai de l'Hôtel de Ville NO-SE,100036719,18 quai de l'Hôtel de Ville,2021-08-10 01:00:00,2017-07-12,Y2H19027732,48.853720,2.357020,1.9,...,False,0,False,False,284,84,6.0,21167,0.0,0.0
33,100036719-103036719,18 quai de l'Hôtel de Ville SE-NO,100036719,18 quai de l'Hôtel de Ville,2021-08-10 01:00:00,2017-07-12,Y2H19027732,48.853720,2.357020,1.9,...,False,0,False,False,284,84,6.0,21167,0.0,0.0
34,100063175-353277233,20 Avenue de Clichy NO-SE,100063175,20 Avenue de Clichy,2021-08-10 01:00:00,2020-07-22,Y2H20073268,48.885290,2.326660,1.9,...,False,0,False,False,284,84,6.0,21167,0.0,0.0


In [7]:
X_final_test_plus = add_external_data(X_final_test)
X_final_test_plus.head()

,counter_id,counter_name,site_id,site_name,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,...,is_holiday,is_weekend,is_lockdown,is_curfew,hosp,rea,incid_rea,rad,Max_Grav_accidents,Count_accidents
0,100049407-353255860,152 boulevard du Montparnasse E-O,100049407,152 boulevard du Montparnasse,2021-09-10 01:00:00,2018-12-07,"48.840801,2.333233",Y2H19070373,48.840801,2.333233,...,False,0,False,False,365,126,5.0,21675,0.0,0.0
43,100049407-353255859,152 boulevard du Montparnasse O-E,100049407,152 boulevard du Montparnasse,2021-09-10 01:00:00,2018-12-07,"48.840801,2.333233",Y2H19070373,48.840801,2.333233,...,False,0,False,False,365,126,5.0,21675,0.0,0.0
31,100036719-104036719,18 quai de l'Hôtel de Ville NO-SE,100036719,18 quai de l'Hôtel de Ville,2021-09-10 01:00:00,2017-07-12,"48.85372,2.35702",Y2H19027732,48.853720,2.357020,...,False,0,False,False,365,126,5.0,21675,0.0,0.0
32,100036719-103036719,18 quai de l'Hôtel de Ville SE-NO,100036719,18 quai de l'Hôtel de Ville,2021-09-10 01:00:00,2017-07-12,"48.85372,2.35702",Y2H19027732,48.853720,2.357020,...,False,0,False,False,365,126,5.0,21675,0.0,0.0
33,100063175-353277233,20 Avenue de Clichy NO-SE,100063175,20 Avenue de Clichy,2021-09-10 01:00:00,2020-07-22,"48.88529,2.32666",Y2H20073268,48.885290,2.326660,...,False,0,False,False,365,126,5.0,21675,0.0,0.0


##  Preprocessing

In [8]:
def _encode_dates(X):
    
    '''
    Splits the 'date' columns of the input DataFrame into several columns (year, month, day, weekday, hour)
    
    Parameters:
        X (pd.DataFrame): the dataframe to modify
    
    Returns:
        X (pd.DataFrame): the modified dataframe
    '''
    
    # Duplicate X to work on it
    X = X.copy()
    
    # Create new columns with date parts from X.date
    X.loc[:, "year"] = X["date"].dt.year
    X.loc[:, "month"] = X["date"].dt.month
    X.loc[:, "day"] = X["date"].dt.day
    X.loc[:, "weekday"] = X["date"].dt.weekday
    X.loc[:, "hour"] = X["date"].dt.hour

    # Adding cosinus and sinus features from date variables to enhance the date periodicity
    # X['cos_hour'] = np.cos(X['hour']*(2.*np.pi/24))
    # X['sin_hour'] = np.sin(X['hour']*(2.*np.pi/24))
    # X['cos_day'] = np.cos(X['day']*(2.*np.pi/30))
    # X['sin_day'] = np.sin(X['day']*(2.*np.pi/30))
    # X['cos_month'] = np.cos(X['month']*(2.*np.pi/12))
    # X['sin_month'] = np.sin(X['month']*(2.*np.pi/12))
    # X['cos_weekday'] = np.cos(X['weekday']*(2.*np.pi/7))
    # X['sin_weekday'] = np.sin(X['weekday']*(2.*np.pi/7))
    
    # Clean the new dataframe and return it
    X.drop(columns=["date"], inplace=True)
    #X.drop(columns=["year", 'month', 'day', 'weekday', 'hour'], inplace=True)
    return X

# Training

## Training without pipeline

In [9]:
def get_RMSE_local(model, X_train, y_train, X_test, y_test):

    # Fit the pipeline on the training data
    model.fit(X_train, y_train)

    # Predict on training and test data
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Calculate RMSE for training and test data
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

    print(f"Train set RMSE: {rmse_train:.2f}")
    print(f"Test set RMSE: {rmse_test:.2f}")

    return rmse_train, rmse_test


### Select features for training

In [10]:
['is_holiday', 'is_weekend', 'is_lockdown', 'hosp', 'rea', 'Count_accidents']
0.9289002341496675

best_RMSE_test: 0.93
best_features_sub: ['is_holiday', 'hosp', 'rea', 'incid_rea']

best_RMSE_test: 0.92
best_features_sub: ['is_holiday', 'hosp', 'rea', 'rad']

best_RMSE_test: 0.44
BEST 0.62 LEADER_BOARD
best_features_sub: ['is_holiday', 'hosp', 'is_lockdown']

SyntaxError: invalid syntax (3277527305.py, line 11)

In [12]:
fixed_features = ['date', "counter_name", "site_id", "t", "u", "rr3"]

not_fixed_features = ["counter_id", "counter_technical_id", "counter_installation_date",
                      'site_name'
                       'counter_installation_date', 
                      'counter_technical_id', 
                      'latitude',
                       'longitude', 
                      'ff', # the wind speed
                      'u', # the humidity
                      'ssfrai', # the fresh snowfall amount
                      'n', # the amount of cloud cover
                      'vv', # the visibility
                      'rr3', # the precipitation amount over 3 hours
                      't', # the temperature
                      'is_holiday', # is holidays
                       'is_weekend', # is week end
                      'is_lockdown', # Lockdown for COVID
                      'is_curfew', # Curfew for COVID
                      'hosp', # Number of patients currently hospitalised for COVID-19
                      'rea', # Number of patients currently in intensive care.
                      'incid_rea', # Number of new patients admitted to intensive care in the last 24 hours.
                       'rad', # Cumulative number of patients hospitalised for COVID-19 who have returned home due to an improvement in their state of health
                      'Max_Grav_accidents', # The maximum severity of all cyclists accidents at a given hour
                      'Count_accidents' # the number of accidents at a given hour in Paris
                     ]

chosen_not_fixed_features = [ 'latitude',
                              'longitude',
                              'ff', # the wind speed
                              'u', # the humidity
                              'ssfrai', # the fresh snowfall amount
                              'n', # the amount of cloud cover
                              'vv', # the visibility
                              'rr3', # the precipitation amount over 3 hours
                              't', # the temperature
                              'is_holiday', # is holidays
                               'is_weekend', # is week end
                              'is_lockdown', # Lockdown for COVID
                              'is_curfew', # Curfew for COVID
                              'hosp', # Number of patients currently hospitalised for COVID-19
                              'rea', # Number of patients currently in intensive care.
                              'incid_rea', # Number of new patients admitted to intensive care in the last 24 hours.
                               'rad', # Cumulative number of patients hospitalised for COVID-19 who have returned home due to an improvement in their state of health
                              'Max_Grav_accidents', # The maximum severity of all cyclists accidents at a given hour
                              'Count_accidents' # the number of accidents at a given hour in Paris
                            ]

chosen_fixed_features = [ 'ff', # the wind speed
                          'u', # the humidity
                          'n', # the amount of cloud cover
                          'vv', # the visibility
                          'rr3', # the precipitation amount over 3 hours
                          't', # the temperature
                          'is_holiday', # is holidays
                           'is_weekend', # is week end
                          'is_lockdown', # Lockdown for COVID
                          'is_curfew', # Curfew for COVID
                          'hosp', # Number of patients currently hospitalised for COVID-19
                          'rea', # Number of patients currently in intensive care.
                          'incid_rea', # Number of new patients admitted to intensive care in the last 24 hours.
                           'rad', # Cumulative number of patients hospitalised for COVID-19 who have returned home due to an improvement in their state of health
                          'Max_Grav_accidents', # The maximum severity of all cyclists accidents at a given hour
                          'Count_accidents' # the number of accidents at a given hour in Paris
                        ]

chosen_fixed_features = ['is_holiday', 'rea', 'rad', 'hosp']

#chosen_fixed_features = ['Count_accidents']

chosen_variables = fixed_features + chosen_fixed_features


X_train_plus_chosen = X_train_plus[chosen_variables]
X_test_plus_chosen = X_test_plus[chosen_variables]
X_final_test_plus_chosen = X_final_test_plus[chosen_variables]

In [13]:
# Identifying continuous variables (float type or int with wide range)

# Initialize the scaler
scaler = StandardScaler()

X_train_plus_chosen_FI = _encode_dates(X_train_plus_chosen)
#X_train_plus_chosen_FI = pd.get_dummies(X_train_plus_chosen_FI, columns=['site_name'])
X_train_plus_chosen_FI = pd.get_dummies(X_train_plus_chosen_FI, columns=['counter_name'])
X_train_plus_chosen_FI = pd.get_dummies(X_train_plus_chosen_FI, columns=['site_id'])
#continuous_columns  = X_train_plus_chosen_FI.select_dtypes(include=['float64', 'float32', 'int64', 'int32']).columns.tolist()
#X_train_plus_chosen_FI[continuous_columns] = scaler.fit_transform(X_train_plus_chosen_FI[continuous_columns])

X_test_plus_chosen_FI = _encode_dates(X_test_plus_chosen)
#X_test_plus_chosen_FI = pd.get_dummies(X_test_plus_chosen_FI, columns=['site_name'])
X_test_plus_chosen_FI = pd.get_dummies(X_test_plus_chosen_FI, columns=['counter_name'])
X_test_plus_chosen_FI = pd.get_dummies(X_test_plus_chosen_FI, columns=['site_id'])
#X_test_plus_chosen_FI[continuous_columns] = scaler.fit_transform(X_test_plus_chosen_FI[continuous_columns])

X_final_test_plus_chosen_FI = _encode_dates(X_final_test_plus_chosen)
#X_final_test_plus_chosen_FI = pd.get_dummies(X_final_test_plus_chosen_FI, columns=['site_name'])
X_final_test_plus_chosen_FI = pd.get_dummies(X_final_test_plus_chosen_FI, columns=['counter_name'])
X_final_test_plus_chosen_FI = pd.get_dummies(X_final_test_plus_chosen_FI, columns=['site_id'])
#X_final_test_plus_chosen_FI[continuous_columns] = scaler.fit_transform(X_final_test_plus_chosen_FI[continuous_columns])

In [31]:
# LGMBRegressor
params = {
        'colsample_bytree': 0.7, 
        'learning_rate': 0.01,
        'max_depth': 11,
        'min_child_samples': 198,
        'min_child_weight': 0.1,
        'n_estimators': 2000,
        'num_leaves': 99,
        'reg_alpha': 1, 
        'reg_lambda': 0.1,
        'subsample': 0.5
}

# params = {'boosting_type': 'gbdt',
#  'class_weight': None,
#  'colsample_bytree': 0.6,
#  'importance_type': 'split',
#  'learning_rate': 0.05,
#  'max_depth': 11,
#  'min_child_samples': 20,
#  'min_child_weight': 0.001,
#  'min_split_gain': 0.0,
#  'n_estimators': 1800,
#  'n_jobs': None,
#  'num_leaves': 96,
#  'objective': None,
#  'random_state': None,
#  'reg_alpha': 0.0,
#  'reg_lambda': 0.0,
#  'subsample': 0.45,
#  'subsample_for_bin': 200000,
#  'subsample_freq': 0}

Regressor = lgb.LGBMRegressor(**params)

#Regressor = xgb.XGBRegressor(objectives = 'reg:squarederror')

# Initialize the XGBRegressor
# Regressor = xgb.XGBRegressor(objectives = 'reg:squarederror',
#  learning_rate= 0.01,
#  max_depth = 15,
#  n_estimators = 600,
#  max_iter = 20,
#  verbose = True,
#  early_stopping = True)

# Train model with selected features
Regressor.fit(X_train_plus_chosen_FI, y_train)

get_RMSE_local(Regressor, X_train_plus_chosen_FI, y_train, X_test_plus_chosen_FI, y_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007923 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1364
[LightGBM] [Info] Number of data points in the train set: 455163, number of used features: 98
[LightGBM] [Info] Start training from score 3.048589
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

(0.37861789907281823, 0.4717446883843572)

### GRIDSEARCH 

#### XGBOOST

In [14]:
from sklearn.model_selection import GridSearchCV, train_test_split

# Splitting for validation
X_train_cv, X_val_cv, y_train_cv, y_val_cv = train_test_split(X_train_plus_chosen_FI, y_train, test_size=0.2, random_state=42)

# Initialize the XGBRegressor
Regressor = xgb.XGBRegressor(objective='reg:squarederror',
                             verbose=True,
                             early_stopping=True
                            )

# Define hyperparameter grid (example)
param_grid = {
    'n_estimators': [700, 900, 1100, 1300, 1500],
    'learning_rate': [0.001, 0.005, 0.01, 0.02],
    'max_depth': [9, 12, 15],
    'subsample': [0.7, 0.8]
}

# Randomized search with cross-validation
randomized_search = RandomizedSearchCV(Regressor, param_grid, n_iter=50, cv=2, scoring='neg_mean_squared_error', verbose = True, n_jobs=-1)
randomized_search.fit(X_train_cv, y_train_cv, eval_set=[(X_val_cv, y_val_cv)], verbose=True)

# Best estimator
best_regressor = randomized_search.best_estimator_

# Evaluate the model using your custom function
# Assuming get_RMSE_local_wt_pipe is your evaluation function
get_RMSE_local(best_regressor, X_train_plus_chosen_FI, y_train, X_test_plus_chosen_FI, y_test)

full_params = best_regressor.get_params()
print("Full Parameters of the Best Regressor:", full_params)

Fitting 2 folds for each of 50 candidates, totalling 100 fits
[0]	validation_0-rmse:1.65013
[1]	validation_0-rmse:1.62417


C:\Users\Thomas\anaconda3\envs\bikes-count\lib\site-packages\xgboost\core.py:160: UserWarning: [12:42:00] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "early_stopping", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


[2]	validation_0-rmse:1.59883
[3]	validation_0-rmse:1.57412
[4]	validation_0-rmse:1.54993
[5]	validation_0-rmse:1.52634
[6]	validation_0-rmse:1.50332
[7]	validation_0-rmse:1.48141
[8]	validation_0-rmse:1.45946
[9]	validation_0-rmse:1.43870
[10]	validation_0-rmse:1.41800
[11]	validation_0-rmse:1.39824
[12]	validation_0-rmse:1.37852
[13]	validation_0-rmse:1.35967
[14]	validation_0-rmse:1.34084
[15]	validation_0-rmse:1.32254
[16]	validation_0-rmse:1.30491
[17]	validation_0-rmse:1.28709
[18]	validation_0-rmse:1.27050
[19]	validation_0-rmse:1.25387
[20]	validation_0-rmse:1.23816
[21]	validation_0-rmse:1.22229
[22]	validation_0-rmse:1.20645
[23]	validation_0-rmse:1.19197
[24]	validation_0-rmse:1.17715
[25]	validation_0-rmse:1.16240
[26]	validation_0-rmse:1.14795
[27]	validation_0-rmse:1.13479
[28]	validation_0-rmse:1.12109
[29]	validation_0-rmse:1.10858
[30]	validation_0-rmse:1.09560
[31]	validation_0-rmse:1.08310
[32]	validation_0-rmse:1.07155
[33]	validation_0-rmse:1.05983
[34]	validation_

C:\Users\Thomas\anaconda3\envs\bikes-count\lib\site-packages\xgboost\core.py:160: UserWarning: [12:43:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "early_stopping", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


Train set RMSE: 0.25
Test set RMSE: 0.48
Full Parameters of the Best Regressor: {'objective': 'reg:squarederror', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.02, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 12, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': 1300, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': 0.7, 'tree_method': None, 'validate_parameters': None, 'verbosity': None, 'verbose': True, 'earl

### LightGBM

In [ ]:
X_train_plus_chosen_FI.head()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from lightgbm import LGBMRegressor
import numpy as np

# Define your X_train_plus_chosen_FI, y_train, X_test_plus_chosen_FI, y_test here

# Splitting for validation
X_train_cv, X_val_cv, y_train_cv, y_val_cv = train_test_split(X_train_plus_chosen_FI, y_train, test_size=0.2, random_state=42)

# Initialize the LightGBM Regressor
Regressor = LGBMRegressor()

# Define hyperparameter grid
param_grid = {
    'colsample_bytree': np.linspace(0.6, 0.8, 2),
    'learning_rate': [0.001, 0.005, 0.01, 0.03, 0.05],
    'max_depth': [9, 10, 11, 12, 13],
    'n_estimators': [1500, 1800, 1900, 2000, 2100, 2200],
    'num_leaves': np.arange(90, 110, 3),
    'subsample': np.linspace(0.45, 0.8, 2)
}

# Randomized search with cross-validation
randomized_search = RandomizedSearchCV(Regressor, param_grid, n_iter=70, cv=2, scoring='neg_mean_squared_error', verbose=True)
randomized_search.fit(X_train_cv, y_train_cv, eval_set=[(X_val_cv, y_val_cv)])

# Best estimator
best_regressor = randomized_search.best_estimator_

# Assuming get_RMSE_local is your evaluation function
# get_RMSE_local(best_regressor, X_train_plus_chosen_FI, y_train, X_test_plus_chosen_FI, y_test)

# Get full parameters of the best regressor
full_params = best_regressor.get_params()
print("Full Parameters of the Best Regressor:", full_params)


Fitting 2 folds for each of 70 candidates, totalling 140 fits
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004240 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1365
[LightGBM] [Info] Number of data points in the train set: 182065, number of used features: 98
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 3.045665
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003312 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1366
[LightGBM] [Info] Number of dat

In [34]:
best_regressor.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 0.6,
 'importance_type': 'split',
 'learning_rate': 0.05,
 'max_depth': 11,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 1800,
 'n_jobs': None,
 'num_leaves': 96,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'subsample': 0.45,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

### Feature selection

In [37]:
all_combination_done = []

In [38]:
def feature_selection(combinations_alr_done):

    # Initialize the scaler
    scaler = StandardScaler()
    best_RMSE_test = 10
    best_features_sub = []

    fixed_features = ['date', "counter_name", "site_id", 't', 'u', 'rr3']
    
    chosen_fixed_features = [ 'ff', # the wind speed
                              'u', # the humidity
                              'n', # the amount of cloud cover
                              'vv', # the visibility
                              'rr3', # the precipitation amount over 3 hours
                              't', # the temperature
                              'is_holiday', # is holidays
                               'is_weekend', # is week end
                              'is_lockdown', # Lockdown for COVID
                              'is_curfew', # Curfew for COVID
                              'hosp', # Number of patients currently hospitalised for COVID-19
                              'rea', # Number of patients currently in intensive care.
                              'incid_rea', # Number of new patients admitted to intensive care in the last 24 hours.
                               'rad', # Cumulative number of patients hospitalised for COVID-19 who have returned home due to an improvement in their state of health
                              'Max_Grav_accidents', # The maximum severity of all cyclists accidents at a given hour
                              'Count_accidents' # the number of accidents at a given hour in Paris
                            ]
    
    chosen_fixed_features = [ 'is_holiday', # is holidays
                              'is_weekend', # is week end
                              'is_lockdown', # Lockdown for COVID
                              'is_curfew', # Curfew for COVID
                              'Max_Grav_accidents', # The maximum severity of all cyclists accidents at a given hour
                              'hosp', # Number of patients currently hospitalised for COVID-19
                              'rea', # Number of patients currently in intensive care.
                              'incid_rea', # Number of new patients admitted to intensive care in the last 24 hours.
                              'rad', # Cumulative number of patients hospitalised for COVID-19 who have returned home due to an improvement in their state of health
                              'Count_accidents' # the number of accidents at a given hour in Paris

                            ]
    
    all_combinations = [list(combo) for r in range(1, len(chosen_fixed_features) + 1) for combo in combinations(chosen_fixed_features, r)]
    combinations_to_test = [item for item in all_combinations if item not in combinations_alr_done]
    
    print(f"{len(combinations_to_test)} combinations of features to test")
    print(f"List of all the combinations to test : {combinations_to_test}")
        
    for i in range(len(combinations_to_test)):

        # -------------------------------------------------- #
        # Selection the features of our training and test set
        # -------------------------------------------------- #
        combination_feature = combinations_to_test[i] 
        chosen_variables = fixed_features + combination_feature
    
        X_train_plus_chosen = X_train_plus[chosen_variables]
        X_test_plus_chosen = X_test_plus[chosen_variables]
        X_final_test_plus_chosen = X_final_test_plus[chosen_variables]
        
        # --------------------------------------- #
        # Preprocessing according to our features
        # --------------------------------------- #
        X_train_plus_chosen_FI = _encode_dates(X_train_plus_chosen)
        #X_train_plus_chosen_FI = pd.get_dummies(X_train_plus_chosen_FI, columns=['site_name'])
        X_train_plus_chosen_FI = pd.get_dummies(X_train_plus_chosen_FI, columns=['counter_name'])
        X_train_plus_chosen_FI = pd.get_dummies(X_train_plus_chosen_FI, columns=['site_id'])
        
        X_test_plus_chosen_FI = _encode_dates(X_test_plus_chosen)
        #X_test_plus_chosen_FI = pd.get_dummies(X_test_plus_chosen_FI, columns=['site_name'])
        X_test_plus_chosen_FI = pd.get_dummies(X_test_plus_chosen_FI, columns=['counter_name'])
        X_test_plus_chosen_FI = pd.get_dummies(X_test_plus_chosen_FI, columns=['site_id'])
        
        X_final_test_plus_chosen_FI = _encode_dates(X_final_test_plus_chosen)
        #X_final_test_plus_chosen_FI = pd.get_dummies(X_final_test_plus_chosen_FI, columns=['site_name'])
        X_final_test_plus_chosen_FI = pd.get_dummies(X_final_test_plus_chosen_FI, columns=['counter_name'])
        X_final_test_plus_chosen_FI = pd.get_dummies(X_final_test_plus_chosen_FI, columns=['site_id'])
            
        # ------- #
        # TRAINING
        # ------- #
        Regressor = xgb.XGBRegressor(objectives = 'reg:squarederror',
         verbose = True)

        # ------- #
        # TESTING
        # ------- #

        RMSE_train, RMSE_test = get_RMSE_local(Regressor, X_train_plus_chosen_FI, y_train, X_test_plus_chosen_FI, y_test)
        print(f"Tested on combination {combination_feature}")
        
        if RMSE_test < best_RMSE_test:
            best_RMSE_test = RMSE_test
            best_features_sub = combination_feature
            print(f"best_RMSE_test updated: {best_RMSE_test:.2f}")
            print(f"best_features_sub updated: {best_features_sub}")

    print(f"best_RMSE_test: {best_RMSE_test:.2f}")
    print(f"best_features_sub: {best_features_sub}")

    return best_RMSE_test, best_features_sub, all_combinations

In [39]:
### Test all combinations and retrieve the combinations tested
best_RMSE_test, best_features_sub, tested_combination = feature_selection(all_combination_done)

# Add the combinations done previously to our combinations_list to not repeat them after
all_combination_done += tested_combination

1023 combinations of features to test
List of all the combinations to test : [['is_holiday'], ['is_weekend'], ['is_lockdown'], ['is_curfew'], ['Max_Grav_accidents'], ['hosp'], ['rea'], ['incid_rea'], ['rad'], ['Count_accidents'], ['is_holiday', 'is_weekend'], ['is_holiday', 'is_lockdown'], ['is_holiday', 'is_curfew'], ['is_holiday', 'Max_Grav_accidents'], ['is_holiday', 'hosp'], ['is_holiday', 'rea'], ['is_holiday', 'incid_rea'], ['is_holiday', 'rad'], ['is_holiday', 'Count_accidents'], ['is_weekend', 'is_lockdown'], ['is_weekend', 'is_curfew'], ['is_weekend', 'Max_Grav_accidents'], ['is_weekend', 'hosp'], ['is_weekend', 'rea'], ['is_weekend', 'incid_rea'], ['is_weekend', 'rad'], ['is_weekend', 'Count_accidents'], ['is_lockdown', 'is_curfew'], ['is_lockdown', 'Max_Grav_accidents'], ['is_lockdown', 'hosp'], ['is_lockdown', 'rea'], ['is_lockdown', 'incid_rea'], ['is_lockdown', 'rad'], ['is_lockdown', 'Count_accidents'], ['is_curfew', 'Max_Grav_accidents'], ['is_curfew', 'hosp'], ['is_cur

C:\Users\Thomas\anaconda3\envs\bikes-count\lib\site-packages\xgboost\core.py:160: UserWarning: [02:22:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "objectives", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


Train set RMSE: 0.43
Test set RMSE: 0.50
Tested on combination ['is_holiday']
best_RMSE_test updated: 0.50
best_features_sub updated: ['is_holiday']


C:\Users\Thomas\anaconda3\envs\bikes-count\lib\site-packages\xgboost\core.py:160: UserWarning: [02:22:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "objectives", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


Train set RMSE: 0.44
Test set RMSE: 0.48
Tested on combination ['is_weekend']
best_RMSE_test updated: 0.48
best_features_sub updated: ['is_weekend']


...

Train set RMSE: 0.41
Test set RMSE: 0.50
Tested on combination ['is_holiday', 'is_lockdown', 'is_curfew', 'Max_Grav_accidents', 'hosp', 'rea', 'incid_rea', 'rad', 'Count_accidents']


C:\Users\Thomas\anaconda3\envs\bikes-count\lib\site-packages\xgboost\core.py:160: UserWarning: [04:13:38] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "objectives", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


Train set RMSE: 0.41
Test set RMSE: 0.52
Tested on combination ['is_weekend', 'is_lockdown', 'is_curfew', 'Max_Grav_accidents', 'hosp', 'rea', 'incid_rea', 'rad', 'Count_accidents']


C:\Users\Thomas\anaconda3\envs\bikes-count\lib\site-packages\xgboost\core.py:160: UserWarning: [04:13:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "objectives", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


Train set RMSE: 0.41
Test set RMSE: 0.50
Tested on combination ['is_holiday', 'is_weekend', 'is_lockdown', 'is_curfew', 'Max_Grav_accidents', 'hosp', 'rea', 'incid_rea', 'rad', 'Count_accidents']
best_RMSE_test: 0.45
best_features_sub: ['is_holiday', 'is_lockdown', 'hosp']


In [40]:
# To store best results after each modification of feature selection
all_combinations = all_combination_done
best_features = best_features_sub
best_RMSE = best_RMSE_test

print(len(all_combinations))
print(best_features)
print(best_RMSE)

1023
['is_holiday', 'is_lockdown', 'hosp']
0.44637931519684115


### Final csv file with current model

#### Train on whole dataset train and test to get the last month

In [17]:
concatenated_X_train_test = pd.concat([X_train_plus_chosen_FI, X_test_plus_chosen_FI], ignore_index=False)
concatenated_y_train_test = np.concatenate([y_train, y_test], axis=0)

best_params_062 = {'objective': 'reg:squarederror', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.01, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 12, 'max_leaves': None, 'min_child_weight': None, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': 1100, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': 0.8, 'tree_method': None, 'validate_parameters': None, 'verbosity': None, 'verbose': True, 'early_stopping': True}

params_060 = {'objective': 'reg:squarederror', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.02, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 12, 'max_leaves': None, 'min_child_weight': None, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': 1300, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': 0.7, 'tree_method': None, 'validate_parameters': None, 'verbosity': None, 'verbose': True, 'early_stopping': True}

Regressor = xgb.XGBRegressor(**params)

# Initialize the MLPRegressor
# Regressor = MLPRegressor(hidden_layer_sizes=(100,), 
#                          activation='relu', 
#                          solver='adam', 
#                          alpha=0.0001, 
#                          batch_size='auto', 
#                          learning_rate='constant', 
#                          learning_rate_init=0.001, 
#                          max_iter=100, 
#                          shuffle=True, 
#                          random_state=None, 
#                          tol=0.0001, 
#                          verbose=True, 
#                          warm_start=True
#                          )

# Train model with selected features
Regressor.fit(concatenated_X_train_test, concatenated_y_train_test)

C:\Users\Thomas\anaconda3\envs\bikes-count\lib\site-packages\xgboost\core.py:160: UserWarning: [12:51:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "early_stopping", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping=True,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.02, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=12,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=1300,
             n_jobs=None, num_parallel_tree=None, ...)

In [18]:
# Get submission kaggle to csv
original_index = X_final_test.index
submit =submission_kaggle(Regressor, X_final_test_plus_chosen_FI, original_index)
print(submit)

[1.3625857 1.1927387 1.2741632 ... 3.480119  2.427347  2.3895016]
          Id  log_bike_count
0      17081        1.362586
1      18655        1.192739
2       3124        1.274163
3       4147        1.139552
4      48210        2.525553
...      ...             ...
51435  42131        3.700244
51436  43042        3.107997
51437  43929        3.480119
51438   5707        2.427347
51439   6855        2.389502

[51440 rows x 2 columns]


### Feature selections with RFECV

In [112]:
from sklearn.feature_selection import RFECV
import xgboost as xgb

# Initialize the XGBRegressor
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror')

# Initialize RFECV
selector = RFECV(estimator=xgb_reg, step=1, cv=2)

# Fit RFECV
selector = selector.fit(X_train_plus_chosen_FI, y_train)

# Print the optimal number of features
print("Optimal number of features : %d" % selector.n_features_)

Optimal number of features : 92


In [114]:
# Print selected features
selected_features = [feature for feature, selected in zip(X_train_plus_chosen_FI.columns, selector.support_) if selected]
print("Selected features:", selected_features)

Selected features: ['latitude', 'longitude', 'ff', 'u', 'n', 'vv', 'rr3', 't', 'is_holiday', 'is_weekend', 'is_lockdown', 'is_curfew', 'hosp', 'rea', 'incid_rea', 'rad', 'Max_Grav_accidents', 'Count_accidents', 'month', 'day', 'weekday', 'hour', 'counter_name_152 boulevard du Montparnasse E-O', 'counter_name_152 boulevard du Montparnasse O-E', "counter_name_18 quai de l'Hôtel de Ville NO-SE", 'counter_name_20 Avenue de Clichy NO-SE', 'counter_name_20 Avenue de Clichy SE-NO', 'counter_name_254 rue de Vaugirard NE-SO', 'counter_name_254 rue de Vaugirard SO-NE', 'counter_name_27 quai de la Tournelle NO-SE', 'counter_name_27 quai de la Tournelle SE-NO', 'counter_name_28 boulevard Diderot E-O', 'counter_name_28 boulevard Diderot O-E', 'counter_name_36 quai de Grenelle NE-SO', 'counter_name_36 quai de Grenelle SO-NE', 'counter_name_38 rue Turbigo NE-SO', 'counter_name_38 rue Turbigo SO-NE', 'counter_name_39 quai François Mauriac NO-SE', 'counter_name_39 quai François Mauriac SE-NO', 'counter

In [116]:
# Transform training and testing sets
X_train_selected = selector.transform(X_train_plus_chosen_FI)
X_test_selected = selector.transform(X_test_plus_chosen_FI)

# Train model with selected features
xgb_reg.fit(X_train_selected, y_train)

get_RMSE_local(xgb_reg, X_train_selected, y_train, X_test_selected, y_test)

Train set RMSE: 0.41
Test set RMSE: 0.95
